In [1]:
from config import Config
import pandas as pd
import numpy as np

In [2]:
import pytorch_lightning as pl
from models.models_new import AttentionModel,GRUModel,GRUModel2,AttentionModelConv1d
from torch.utils.data import DataLoader

In [3]:
from dataloader.np_loader import NumpyDataDatasetInference

In [4]:
cfg = Config()

In [5]:
class ModelTrainerInference(pl.LightningModule):
    def __init__(self, cfg, num_workers = 15):
        super().__init__()
        
        # self.model = AttentionModel(cfg.embed_dim, cfg.feat_dim, cfg.num_heads, cfg.ff_dim, cfg.dropout_rate, cfg.num_blocks)
        # self.model = GRUModel2(cfg.embed_dim, cfg.feat_dim, cfg.dropout_rate, cfg.num_layers_rnn)
        self.model = AttentionModelConv1d(cfg.embed_dim, cfg.feat_dim, cfg.num_heads, cfg.ff_dim, cfg.dropout_rate, cfg.num_blocks)
        
        self.batch_size = cfg.batch_size
        
        self.num_workers = num_workers
        

    def forward(self, x_cont):
        return self.model(x_cont)
        

In [6]:
def predict_data(X_p, model):
    p_dataset = NumpyDataDatasetInference(X_p)
    p_loader = DataLoader(dataset=p_dataset, batch_size=512, shuffle=False, num_workers=15)
    preds = []
    for step, x in enumerate(p_loader):
        y_hat = model(x).tolist()
        preds.extend(y_hat)
    return np.array(preds)
        
    

In [7]:
start = 0; end = 0
sub = pd.read_csv(f'{cfg.data_dir}sample_submission.csv')

# REARANGE SUB ROWS TO MATCH 20 TEST FILES
# sub['hash'] = sub['customer_ID'].str[-16:].str.hex_to_int().astype('int64')
sub['hash'] = sub['customer_ID'].str[-16:].apply(lambda x: int(x, 16)).astype('int64')
test_hash_index = np.load(f'{cfg.prc_data_dir}test_hashes_data.npy')
sub = sub.set_index('hash').loc[test_hash_index].reset_index(drop=True)

for k in range(20):
    print(f'Inferring Test_File_{k+1}')
    X_test = np.load(f'{cfg.prc_data_dir}test_data_{k+1}.npy')
    end = start + X_test.shape[0]

    # INFER 5 FOLD MODELS f"multihead_attention_{fold}"
    # print(f'checkpoints/{cfg.model_name}_{0}_{cfg.version}.ckpt')
    model = ModelTrainerInference.load_from_checkpoint(f'checkpoints/{cfg.model_name}_{0}_{cfg.version}.ckpt', cfg=cfg)
    model.eval()
    p = predict_data(X_test, model)
    
    for j in range(1,5):
        model = ModelTrainerInference.load_from_checkpoint(f'checkpoints/{cfg.model_name}_{j}_{cfg.version}.ckpt', cfg=cfg)
        model.eval()
        p += predict_data(X_test, model)
    p /= 5.0

    sub.loc[start:end-1,'prediction'] = p
    start = end

Inferring Test_File_1
Inferring Test_File_2
Inferring Test_File_3
Inferring Test_File_4
Inferring Test_File_5
Inferring Test_File_6
Inferring Test_File_7
Inferring Test_File_8
Inferring Test_File_9
Inferring Test_File_10
Inferring Test_File_11
Inferring Test_File_12
Inferring Test_File_13
Inferring Test_File_14
Inferring Test_File_15
Inferring Test_File_16
Inferring Test_File_17
Inferring Test_File_18
Inferring Test_File_19
Inferring Test_File_20


In [8]:
sub.to_csv(f'{cfg.model_name}_{cfg.version}_submission.csv',index=False)
print('Submission file shape is', sub.shape )
display( sub.head() )

Submission file shape is (924621, 2)


,customer_ID,prediction
0,038be0571bd6b3776cb8512731968f4de302c811030124...,0.002081
1,0074a0233ef766b52884608cc8cf9098f59d885b5d59fc...,0.000434
2,060b8b7f30f795a0e93995d45b29461ffa6ece0eeb5c3d...,0.120746
3,03a1d125bdd776000bf0b28238d0bea240ad581d332e70...,0.178329
4,0290f245dd35ba899af52316ccc62b2627e7ae18cd76a2...,0.431587


In [9]:
# import torch
# import torch.nn as nn

In [10]:
# a = torch.rand(2,13,188)

In [11]:
# m1 = nn.Conv1d(13, 7, kernel_size=3, stride=2, padding=3)

In [12]:
# o1 = m1(a)

In [13]:
# o1.shape